In [10]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Dropout, Flatten,Input,Reshape, concatenate
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM,Conv1DTranspose,UpSampling1D,AveragePooling1D
from tensorflow.keras.models import model_from_json
from tensorflow.keras import backend as K 
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix,f1_score,precision_score,recall_score
from sklearn.decomposition import KernelPCA,PCA
from sklearn.preprocessing import StandardScaler
from time import time
# from sklearn.svm import SVC
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
# from sklearn.ensemble import GradientBoostingClassifier
# from sklearn.linear_model import LogisticRegression
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.naive_bayes import BernoulliNB
import pickle
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.optimizers import Adam
# from keras import layers
from tensorflow.keras.models import Sequential 
# from focal_loss import binary_focal_loss
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold 
import xai.xai as xai
import xai.xai.data
import shap
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import tensorflow as tf
tf.__version__

'2.10.0'

In [3]:
df=pd.read_csv("dataset_0-32000_1-25000.csv")

In [4]:
df.head()

,Unnamed: 0,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,BaseExcess,HCO3,...,Phosphate,Potassium,Hct,Hgb,PTT,WBC,Platelets,Age,SepsisLabel,SIRS
0,0,80.147727,97.295455,37.080909,106.068182,59.924318,39.641932,16.625000,-0.022727,25.590909,...,3.415909,4.229545,27.227273,9.212500,28.675000,12.561364,248.795455,77.0,0.0,1.0
1,1,80.867647,97.338235,36.910294,99.294118,62.132647,44.860294,17.426471,-0.514706,25.794118,...,3.385294,4.238235,29.480882,9.967647,43.573529,11.250000,118.676471,79.0,0.0,0.0
2,2,89.638889,97.583333,37.046667,120.194444,77.648333,54.069722,19.916667,5.666667,30.944444,...,3.861111,4.133333,31.594444,10.761111,30.366667,13.483333,406.333333,81.0,0.0,1.0
3,3,81.391892,97.662162,36.869730,114.587838,84.567748,67.534054,19.108108,-0.756757,23.621622,...,2.948649,3.951351,33.808108,11.640541,72.045946,9.929730,153.864865,57.0,0.0,0.0
4,4,98.000000,97.473684,37.222632,110.657895,74.948421,57.816842,20.894737,0.710526,24.157895,...,3.231579,4.126316,28.336842,9.352632,34.463158,18.244737,615.157895,28.0,0.0,3.0


In [5]:
df=df.drop('Unnamed: 0',axis=1)
df.head()

,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,BaseExcess,HCO3,FiO2,...,Phosphate,Potassium,Hct,Hgb,PTT,WBC,Platelets,Age,SepsisLabel,SIRS
0,80.147727,97.295455,37.080909,106.068182,59.924318,39.641932,16.625000,-0.022727,25.590909,0.526136,...,3.415909,4.229545,27.227273,9.212500,28.675000,12.561364,248.795455,77.0,0.0,1.0
1,80.867647,97.338235,36.910294,99.294118,62.132647,44.860294,17.426471,-0.514706,25.794118,0.576471,...,3.385294,4.238235,29.480882,9.967647,43.573529,11.250000,118.676471,79.0,0.0,0.0
2,89.638889,97.583333,37.046667,120.194444,77.648333,54.069722,19.916667,5.666667,30.944444,0.583333,...,3.861111,4.133333,31.594444,10.761111,30.366667,13.483333,406.333333,81.0,0.0,1.0
3,81.391892,97.662162,36.869730,114.587838,84.567748,67.534054,19.108108,-0.756757,23.621622,0.467568,...,2.948649,3.951351,33.808108,11.640541,72.045946,9.929730,153.864865,57.0,0.0,0.0
4,98.000000,97.473684,37.222632,110.657895,74.948421,57.816842,20.894737,0.710526,24.157895,0.513158,...,3.231579,4.126316,28.336842,9.352632,34.463158,18.244737,615.157895,28.0,0.0,3.0


In [6]:
df.SepsisLabel.value_counts()

0.0    32000
1.0    25000
Name: SepsisLabel, dtype: int64

In [7]:
X=df.drop('SepsisLabel',axis=1)
y=df['SepsisLabel']

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size =0.2,stratify=y)

In [12]:
X_train_selected=X_train[['FiO2', 'O2Sat', 'Magnesium', 'SaO2', 'Lactate', 'pH', 'BaseExcess','Potassium',
                          'Temp', 'Calcium', 'Phosphate', 'Creatinine', 'Chloride','HCO3', 'Resp', 'Hgb', 'Hct', 'BUN','SIRS']]
X_test_selected=X_test[['FiO2', 'O2Sat', 'Magnesium', 'SaO2', 'Lactate', 'pH', 'BaseExcess','Potassium',
                        'Temp', 'Calcium', 'Phosphate', 'Creatinine', 'Chloride','HCO3', 'Resp', 'Hgb', 'Hct', 'BUN','SIRS']]

In [13]:
y_train.value_counts(),y_test.value_counts()

(0.0    25600
 1.0    20000
 Name: SepsisLabel, dtype: int64,
 0.0    6400
 1.0    5000
 Name: SepsisLabel, dtype: int64)

In [15]:
pd.set_option('display.max_columns',None)
X_test.head()

,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,BaseExcess,HCO3,FiO2,pH,PaCO2,SaO2,BUN,Calcium,Chloride,Creatinine,Glucose,Lactate,Magnesium,Phosphate,Potassium,Hct,Hgb,PTT,WBC,Platelets,Age,SIRS
22320,101.892857,95.678571,37.127857,82.178571,51.547857,38.464643,25.642857,2.428571,26.857143,0.550000,7.376429,48.785714,90.357143,61.285714,8.507143,99.714286,2.435714,134.571429,2.664286,2.428571,4.400000,4.321429,29.785714,10.207143,32.914286,11.171429,216.642857,68.0,2.0
36459,107.306334,98.870889,37.892278,119.846833,83.411389,66.258222,16.564555,-2.258222,25.153167,0.471772,7.269431,54.363277,91.693666,10.846833,8.384683,110.435445,0.484683,124.153167,1.684683,2.000000,2.656456,4.056456,31.142022,10.841139,32.566078,11.503289,274.693666,45.0,1.0
51397,100.500000,100.000000,37.200000,89.500000,59.000000,41.000000,14.500000,-4.000000,19.000000,0.400000,7.410000,29.000000,98.000000,6.000000,7.800000,117.000000,0.700000,87.000000,1.400000,1.600000,2.300000,3.300000,31.700000,9.800000,37.300000,14.400000,266.000000,74.0,3.0
10110,93.166667,98.083333,36.898333,153.333333,104.500000,80.666667,17.333333,-4.833333,21.000000,0.533333,7.328333,37.000000,96.666667,57.000000,7.850000,101.833333,5.700000,257.000000,2.583333,1.916667,5.050000,4.566667,30.066667,10.000000,32.866667,13.366667,347.000000,31.0,2.0
4918,84.240000,96.220000,36.945200,133.680000,84.359600,60.479400,21.100000,1.720000,24.640000,0.548000,7.381200,46.820000,63.740000,16.520000,8.368000,106.000000,0.888000,79.280000,1.580000,2.000000,3.088000,4.052000,32.908000,11.236000,75.212000,11.628000,254.080000,77.0,1.0


In [16]:
y_test.head()

22320    0.0
36459    1.0
51397    1.0
10110    0.0
4918     0.0
Name: SepsisLabel, dtype: float64

In [17]:
minmax = pickle.load(open('minmax_for_autoencoder.pkl','rb'))
X_train_selected_scaled=minmax.transform(X_train_selected)
X_test_selected_scaled=minmax.transform(X_test_selected)

In [18]:
X_train_selected_scaled.shape,X_test_selected_scaled.shape

((45600, 19), (11400, 19))

In [20]:
X_test_selected_scaled[:5]

array([[0.55      , 0.86059908, 0.22744361, 0.8731203 , 0.08586361,
        0.6520936 , 0.26082251, 0.31128748, 0.78926576, 0.29724702,
        0.28378378, 0.10965795, 0.63284379, 0.51648352, 0.41285714,
        0.26869607, 0.33307312, 0.22749326, 0.5       ],
       [0.47177223, 0.96357707, 0.17105263, 0.89070614, 0.05173113,
        0.55985393, 0.2253165 , 0.27857476, 0.83197082, 0.29086892,
        0.16597673, 0.01806025, 0.73304154, 0.48371475, 0.23129111,
        0.2899711 , 0.35777818, 0.03715786, 0.25      ],
       [0.4       , 1.        , 0.11842105, 0.97368421, 0.04181185,
        0.68103448, 0.21212121, 0.18518519, 0.79329609, 0.26041667,
        0.14189189, 0.02816901, 0.79439252, 0.36538462, 0.19      ,
        0.25503356, 0.36794171, 0.01886792, 0.75      ],
       [0.53333333, 0.93817204, 0.16008772, 0.95614035, 0.08304297,
        0.61063218, 0.20580808, 0.34156379, 0.7764432 , 0.26302083,
        0.3277027 , 0.2629108 , 0.65264798, 0.40384615, 0.24666667,
        0.261

In [14]:
X_train_selected_scaled_cnn=X_train_selected_scaled.reshape(45600, 19,1)
X_test_selected_scaled_cnn=X_test_selected_scaled.reshape(11400, 19,1)
print("Train shape for CNN: ",X_train_selected_scaled_cnn.shape)
print("Test shape for CNN: ",X_test_selected_scaled_cnn.shape)

Train shape for CNN:  (45600, 19, 1)
Test shape for CNN:  (11400, 19, 1)


## AutoEncoder by CNN Encoder and CNN Decoder

In [15]:
input_dim = Input(shape = (19, 1))

# cnn_enc=Sequential()
cnn_enc = layers.Conv1D(64, 19, activation='relu', padding='same')(input_dim)
cnn_enc = layers.Conv1D(32, 19, activation='relu', padding='same')(cnn_enc)
cnn_enc = layers.Conv1D(16, 19, activation='relu', padding='same')(cnn_enc)
cnn_enc = layers.Flatten()(cnn_enc)

bottleneck = Dense(11, activation='sigmoid')(cnn_enc)

reshp= Reshape((11,1))(bottleneck)

cnn_dec=layers.Conv1D(16, 11, activation='relu',padding='same')(reshp)
cnn_dec=layers.Conv1D(32, 11, activation='relu',padding='same')(cnn_dec)
cnn_dec=layers.Conv1D(64, 11, activation='relu',padding='same')(cnn_dec)

dec_flat=layers.Flatten()(cnn_dec)

mlp=Dense(256,activation='relu')(dec_flat)
mlp=Dense(19,activation='relu')(mlp)

autoencoder_cnn_cnn = keras.Model(input_dim, mlp)

print(autoencoder_cnn_cnn.summary())

###### Imp Model

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 19, 1)]           0         
                                                                 
 conv1d (Conv1D)             (None, 19, 64)            1280      
                                                                 
 conv1d_1 (Conv1D)           (None, 19, 32)            38944     
                                                                 
 conv1d_2 (Conv1D)           (None, 19, 16)            9744      
                                                                 
 flatten (Flatten)           (None, 304)               0         
                                                                 
 dense (Dense)               (None, 11)                3355      
                                                                 
 reshape (Reshape)           (None, 11, 1)             0     

### GRAD Cam

In [16]:
def grad_cam(layer_name, data, autoenc, classifa, C_mod=True):
    model1=autoenc
    if C_mod==True:
        model1==classifa
        
    grad_model = tf.keras.models.Model([autoenc.inputs], [model1.get_layer(layer_name).output,
                                                          classifa.output])
    weights_list=model.get_weights()
    grad_model.set_weights(weights_list[0:10])
    last_conv_layer_output, preds = grad_model(data)

    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(data)
        pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    grads = tape.gradient(class_channel, last_conv_layer_output)

    pooled_grads = tf.reduce_mean(grads, axis=(0))

    last_conv_layer_output = last_conv_layer_output[0]

    heatmap = last_conv_layer_output * pooled_grads
    heatmap = tf.reduce_mean(heatmap, axis=(1))
    heatmap = np.expand_dims(heatmap,0)
    return grad_model, heatmap


In [17]:
# data

In [ ]:
layer_name = "<last conv layer name>"
for i in X_test:
    data = np.expand_dims(i,0)
    heatmap = grad_cam(layer_name,data)

    plt.figure(figsize=(30,4))
    plt.imshow(np.expand_dims(heatmap,axis=2),cmap='Reds', aspect="auto", interpolation='nearest',extent=[0,300,i.min(),i.max()], alpha=0.5)
    plt.plot(i,'k')
    plt.colorbar()
    plt.show()

In [18]:
with tf.GradientTape() as tape:
    last_conv_layer_output, preds = grad_model(data)
    pred_index = tf.argmax(preds[0])
    class_channel = preds[:, pred_index]

grads = tape.gradient(class_channel, last_conv_layer_output)

pooled_grads = tf.reduce_mean(grads, axis=(0))

last_conv_layer_output = last_conv_layer_output[0]

heatmap = last_conv_layer_output * pooled_grads
# heatmap = tf.reduce_mean(heatmap, axis=(1))
# heatmap = np.expand_dims(heatmap,0)

NameError: name 'grad_model' is not defined

In [ ]:
class_channel

In [ ]:
last_conv_layer_output.shape

In [ ]:
heatmap.shape

In [ ]:
grads.shape

In [ ]:
last_conv_layer_output, preds = grad_model(data)

In [ ]:
last_conv_layer_output.shape

In [ ]:
preds.shape

In [ ]:
heatmap

In [ ]:
layer_name = "conv1d"
overall_heatmap_for_training=[]
for i in X_train_selected_scaled_cnn:
    data = np.expand_dims(i,0)
    heatmap = grad_cam('conv1d_1','dense',data,autoencoder_cnn_cnn2)
    overall_heatmap_for_training.append(heatmap)
#     plt.figure(figsize=(30,4))
#     plt.imshow(np.expand_dims(heatmap,axis=2),cmap='Reds', aspect="auto", interpolation='nearest', alpha=0.7)
# #         plt.plot(i,'k')
#     plt.colorbar()
#     plt.show()
#     print(heatmap)
#     print(heatmap.shape)


In [ ]:
autoencoder_cnn_cnn.compile(optimizer='adam', loss='mse',metrics=['accuracy','mae','MeanSquaredError'])

In [ ]:
# autoencoder_cnn_cnn.fit(X_train_selected_scaled_cnn,X_train_selected_scaled,epochs=100,validation_split=0.2,verbose=2)

In [ ]:
# autoencoder_cnn_cnn.history.history.keys()

In [ ]:
# plt.plot(autoencoder_cnn_cnn.history.history['accuracy'])
# plt.plot(autoencoder_cnn_cnn.history.history['val_accuracy'])
# plt.title('model accuracy')
# plt.ylabel('accuracy')
# plt.xlabel('epoch')
# plt.legend(['train', 'val'], loc='lower right')
# plt.show()

In [ ]:
autoencoder_cnn_cnn.summary()

In [ ]:
# plt.plot(autoencoder_cnn_cnn.history.history['loss'])
# plt.plot(autoencoder_cnn_cnn.history.history['val_loss'])
# plt.title('model loss')
# plt.ylabel('Loss')
# plt.xlabel('epoch')
# plt.legend(['train', 'val'], loc='lower right')
# plt.show()

In [ ]:
# intermediate_layer_model = Model(inputs=autoencoder_cnn_cnn.input, outputs=autoencoder_cnn_cnn.get_layer(index=5).output)
# X_train_autoencoder_cnn_cnn_11_1 = intermediate_layer_model.predict(X_train_selected_scaled_cnn)
# X_test_autoencoder_cnn_cnn_11_1 = intermediate_layer_model.predict(X_test_selected_scaled_cnn)
# print("Shape of train: ",X_train_autoencoder_cnn_cnn_11_1.shape)
# print("Shape of train: ",X_test_autoencoder_cnn_cnn_11_1.shape)

In [ ]:
ann_cnn_cnn1 = keras.Sequential([
    keras.layers.Dense(128, input_shape=(11,), activation='relu'),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

ann_cnn_cnn1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy',tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])

ann_cnn_cnn1.fit(X_train_autoencoder_cnn_cnn_11_1, y_train, epochs=60,validation_split=0.25)

pred=ann_cnn_cnn1.predict(X_test_autoencoder_cnn_cnn_11_1)
pred1=[]
for j in pred:
    if j[0]>=0.5:
        pred1.append(1)
    else:
        pred1.append(0)
        
print(confusion_matrix(y_test,pred1))
print(classification_report(y_test,pred1))

In [ ]:
[[5835  565]
 [ 284 4716]]

In [ ]:
### New archietecture of the previous model

In [ ]:
input_dim = Input(shape = (19, 1))

# cnn_enc=Sequential()
cnn_enc = layers.Conv1D(64, 19, activation='relu', padding='same')(input_dim)
cnn_enc = layers.Conv1D(32, 19, activation='relu', padding='same')(cnn_enc)
cnn_enc = layers.Conv1D(16, 19, activation='relu', padding='same')(cnn_enc)
cnn_enc = layers.Conv1D(8, 19, activation='relu', padding='same')(cnn_enc)
cnn_enc = layers.Flatten()(cnn_enc)

bottleneck = Dense(11, activation='sigmoid')(cnn_enc)

reshp= Reshape((11,1))(bottleneck)

cnn_dec=layers.Conv1D(8, 11, activation='relu',padding='same')(reshp)
cnn_dec=layers.Conv1D(16, 11, activation='relu',padding='same')(cnn_dec)
cnn_dec=layers.Conv1D(32, 11, activation='relu',padding='same')(cnn_dec)
cnn_dec=layers.Conv1D(64, 11, activation='relu',padding='same')(cnn_dec)

dec_flat=layers.Flatten()(cnn_dec)

# mlp=Dense(256,activation='relu')(dec_flat)
mlp=Dense(19,activation='relu')(dec_flat)

autoencoder_cnn_cnn1 = keras.Model(input_dim, mlp)

print(autoencoder_cnn_cnn1.summary())


In [ ]:
autoencoder_cnn_cnn1.compile(optimizer='adam', loss='mse',metrics=['accuracy','mae','MeanSquaredError'])

In [ ]:
autoencoder_cnn_cnn1.fit(X_train_selected_scaled_cnn,X_train_selected_scaled,epochs=100,validation_split=0.2,verbose=2)

In [ ]:
plt.plot(autoencoder_cnn_cnn1.history.history['accuracy'])
plt.plot(autoencoder_cnn_cnn1.history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='lower right')
plt.show()

In [ ]:
plt.plot(autoencoder_cnn_cnn1.history.history['loss'])
plt.plot(autoencoder_cnn_cnn1.history.history['val_loss'])
plt.title('model accuracy')
plt.ylabel('Loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper right')
plt.show()

In [ ]:
intermediate_layer_model1 = Model(inputs=autoencoder_cnn_cnn1.input, outputs=autoencoder_cnn_cnn1.get_layer(index=6).output)
X_train_autoencoder_cnn_cnn_11_2 = intermediate_layer_model1.predict(X_train_selected_scaled_cnn)
X_test_autoencoder_cnn_cnn_11_2 = intermediate_layer_model1.predict(X_test_selected_scaled_cnn)
print("Shape of train: ",X_train_autoencoder_cnn_cnn_11_2.shape)
print("Shape of train: ",X_test_autoencoder_cnn_cnn_11_2.shape)

In [ ]:
ann_cnn_cnn1 = keras.Sequential([
    keras.layers.Dense(128, input_shape=(11,), activation='relu'),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

ann_cnn_cnn1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy',tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])

ann_cnn_cnn1.fit(X_train_autoencoder_cnn_cnn_11_2, y_train, epochs=60,validation_split=0.25)

pred1=ann_cnn_cnn1.predict(X_test_autoencoder_cnn_cnn_11_2)
pred2=[]
for j in pred1:
    if j[0]>=0.5:
        pred2.append(1)
    else:
        pred2.append(0)
        
print(confusion_matrix(y_test,pred2))
print(classification_report(y_test,pred2))

# Main model

In [ ]:
#### Since there is not such significant difference in the last model after removing one dense layer we are sticking with the 
# first model only and using other loss functions to see the change

### As we are shrinking the layers before and after bottleneck slight misclassification taking place that is lowering the recall

# Another Architecture

In [19]:
input_dim = Input(shape = (19, 1))

# cnn_enc=Sequential()
cnn_enc = layers.Conv1D(128, 19, activation='relu', padding='same')(input_dim)
cnn_enc = layers.Conv1D(64, 19, activation='relu', padding='same')(cnn_enc)
cnn_enc = layers.Conv1D(32, 19, activation='relu', padding='same')(cnn_enc)
cnn_enc = layers.Conv1D(16, 19, activation='relu', padding='same')(cnn_enc)
cnn_enc = layers.Flatten()(cnn_enc)

bottleneck = Dense(11, activation='sigmoid')(cnn_enc)

reshp= Reshape((11,1))(bottleneck)

cnn_dec=layers.Conv1D(16, 11, activation='relu',padding='same')(reshp)
cnn_dec=layers.Conv1D(32, 11, activation='relu',padding='same')(cnn_dec)
cnn_dec=layers.Conv1D(64, 11, activation='relu',padding='same')(cnn_dec)
cnn_dec=layers.Conv1D(128, 11, activation='relu',padding='same')(cnn_dec)

dec_flat=layers.Flatten()(cnn_dec)

# mlp=Dense(256,activation='relu')(dec_flat)
mlp=Dense(19,activation='relu')(dec_flat)

autoencoder_cnn_cnn2 = keras.Model(input_dim, mlp)

print(autoencoder_cnn_cnn2.summary())

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 19, 1)]           0         
                                                                 
 conv1d_6 (Conv1D)           (None, 19, 128)           2560      
                                                                 
 conv1d_7 (Conv1D)           (None, 19, 64)            155712    
                                                                 
 conv1d_8 (Conv1D)           (None, 19, 32)            38944     
                                                                 
 conv1d_9 (Conv1D)           (None, 19, 16)            9744      
                                                                 
 flatten_2 (Flatten)         (None, 304)               0         
                                                                 
 dense_3 (Dense)             (None, 11)                3355

In [ ]:
autoencoder_cnn_cnn2.compile(optimizer='adam', loss='mse',metrics=['accuracy','mae','MeanSquaredError'])

In [ ]:
autoencoder_cnn_cnn2.fit(X_train_selected_scaled_cnn,X_train_selected_scaled,epochs=100,validation_split=0.2,verbose=2)

In [22]:
import os

In [23]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# # Create a callback that saves the model's weights
# cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
#                                                  save_weights_only=True,
#                                                  verbose=1)

# # Train the model with the new callback
# autoencoder_cnn_cnn2.fit(X_train_selected_scaled_cnn, X_train_selected_scaled, epochs=1,validation_split=0.2, verbose=2, 
#           callbacks=[cp_callback])  # Pass callback to training

In [24]:
autoencoder_cnn_cnn2.load_weights(checkpoint_path)

In [ ]:
for layer in autoencoder_cnn_cnn2.layers:
    print(layer.name)

In [ ]:
# Keep the trainable layers only
layer_names=['conv1d_6', 'conv1d_7', 'conv1d_8', 'conv1d_9']

In [ ]:
layer_with_heatmap_dict={}
for layer in layer_names:
    overall_heatmap_for_training=[]
    for i in X_train_selected_scaled_cnn:
        data = np.expand_dims(i,0)
        heatmap = grad_cam(layer,'dense',data,autoencoder_cnn_cnn2)
        overall_heatmap_for_training.append(heatmap)
    layer_with_heatmap_dict[layer]=overall_heatmap_for_training
    print("heatmaps stored for ",layer)

In [ ]:
np.array(layer_with_heatmap_dict["conv1d_3"]).reshape(45600,19).shape

In [ ]:
for key,value in layer_with_heatmap_dict.items():
    heatmap=np.array(layer_with_heatmap_dict[key]).reshape(45600,19)
    heatmap=np.average(heatmap,axis=0).reshape(1,19)
    print("for ",key," heatmap shape ",heatmap.shape)
    plt.figure(figsize=(10,2))
    plt.imshow(np.expand_dims(heatmap,axis=2),cmap='Reds', aspect="auto", interpolation='nearest', alpha=0.7)
#         plt.plot(i,'k')
    plt.colorbar()
    plt.show()

In [ ]:
lst=[]
for key,value in layer_with_heatmap_dict.items():
    heatmap=np.array(layer_with_heatmap_dict[key]).reshape(45600,19)
    heatmap=np.average(heatmap,axis=0).reshape(1,19)
    lst.append(heatmap[0])
    print("for ",key," heatmap shape ",heatmap)

In [ ]:
heatmap=pd.DataFrame(lst).T
heatmap.columns=['EB1','EB2','EB3','EB4']
heatmap.index=['FiO2', 'O2Sat', 'Magnesium', 'SaO2', 'Lactate', 'pH', 'BaseExcess','Potassium',
                          'Temp', 'Calcium', 'Phosphate', 'Creatinine', 'Chloride','HCO3', 'Resp', 'Hgb', 'Hct', 'BUN','SIRS']
heatmap

In [ ]:
# determining the name of the file
file_name = 'Explain_autoencoder.xlsx'
  
# saving the excel
heatmap.to_excel(file_name)
print('DataFrame is written to Excel File successfully.')

In [25]:
intermediate_layer_model11 = Model(inputs=autoencoder_cnn_cnn2.input, outputs=autoencoder_cnn_cnn2.get_layer(index=6).output)
X_train_autoencoder_cnn_cnn_11_12 = intermediate_layer_model11.predict(X_train_selected_scaled_cnn)
X_test_autoencoder_cnn_cnn_11_12 = intermediate_layer_model11.predict(X_test_selected_scaled_cnn)
print("Shape of train: ",X_train_autoencoder_cnn_cnn_11_12.shape)
print("Shape of train: ",X_test_autoencoder_cnn_cnn_11_12.shape)

357/357 [==============================] - 1s 3ms/step
Shape of train:  (45600, 11)
Shape of train:  (11400, 11)


In [ ]:
# KNN
knn=KNeighborsClassifier()
knn.fit(X_train_autoencoder_cnn_cnn_11_12,y_train)
pred=knn.predict(X_test_autoencoder_cnn_cnn_11_12)
print("Accuray: ",accuracy_score(y_test,pred))
print("Precision: ",precision_score(y_test,pred))
print("Recall: ",recall_score(y_test,pred))
print("F1 Score: ",f1_score(y_test,pred))

In [26]:
ann_cnn_cnn2 = tf.keras.Sequential([
    tf.keras.layers.Dense(128, input_shape=(11,), activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

ann_cnn_cnn2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss="binary_crossentropy",
              metrics=['accuracy',tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])

# ann_cnn_cnn2.fit(X_train_autoencoder_cnn_cnn_11_12, y_train, epochs=60,validation_split=0.25)

# pred2=ann_cnn_cnn2.predict(X_test_autoencoder_cnn_cnn_11_12)
# pred3=[]
# for j in pred2:
#     if j[0]>=0.5:
#         pred3.append(1)
#     else:
#         pred3.append(0)
        
# print(confusion_matrix(y_test,pred3))
# print(classification_report(y_test,pred3))

In [ ]:
ann_cnn_cnn2.summary()

In [27]:
checkpoint_path = "training_1_classifier/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# Train the model with the new callback
# ann_cnn_cnn2.fit(X_train_autoencoder_cnn_cnn_11_12, y_train, epochs=1,validation_split=0.2, verbose=2, 
#           callbacks=[cp_callback])  # Pass callback to training

In [28]:
ann_cnn_cnn2.load_weights(checkpoint_path)

In [ ]:


# ann_cnn_cnn2 = keras.Sequential([
#     keras.layers.Dense(128, input_shape=(11,), activation='relu'),
#     keras.layers.Dense(256, activation='relu'),
#     keras.layers.Dense(64, activation='relu'),
#     keras.layers.Dense(1, activation='sigmoid')
# ])

# ann_cnn_cnn2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
#               loss="binary_crossentropy",
#               metrics=['accuracy',tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])

# ann_cnn_cnn2.fit(X_train_autoencoder_cnn_cnn_11_12, y_train, epochs=60,validation_split=0.25)

# pred2=ann_cnn_cnn2.predict(X_test_autoencoder_cnn_cnn_11_12)
# pred3=[]
# for j in pred2:
#     if j[0]>=0.5:
#         pred3.append(1)
#     else:
#         pred3.append(0)
        
# print(confusion_matrix(y_test,pred3))
# print(classification_report(y_test,pred3))

#### SHAP FOR EXPLAINABILTY OF ANN

In [34]:
import tensorflow.tensorflow as tf

ModuleNotFoundError: No module named 'tensorflow.tensorflow'

In [30]:
import tensorflow.keras as keras 

In [31]:
tf.keras.backend.set_learning_phase(True)

`tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.


In [32]:
explainer=shap.DeepExplainer(ann_cnn_cnn2, X_train_autoencoder_cnn_cnn_11_12)

keras is no longer supported, please use tf.keras instead.
Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode. See PR #1483 for discussion.
You have provided over 5k background samples! For better performance consider using smaller random sample.


In [35]:
X_train_autoencoder_cnn_cnn_11_12.shape

(45600, 11)

In [36]:
pd.DataFrame(X_train_autoencoder_cnn_cnn_11_12).to_csv('X_train_autoencoder_cnn_cnn_11_12',index=False)

In [38]:
pd.DataFrame(X_test_autoencoder_cnn_cnn_11_12).to_csv('X_test_autoencoder_cnn_cnn_11_12',index=False)
pd.DataFrame(y_train).to_csv('y_train',index=False)
pd.DataFrame(y_test).to_csv('y_test',index=False)

In [33]:
shap_values_features=explainer.shap_values(X_train_autoencoder_cnn_cnn_11_12)

AttributeError: in user code:

    File "C:\Users\Admin\anaconda3\envs\env2\lib\site-packages\shap\explainers\_deep\deep_tf.py", line 243, in grad_graph  *
        out = self.model(shap_rAnD)
    File "C:\Users\Admin\anaconda3\envs\env2\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler  **
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\Admin\anaconda3\envs\env2\lib\site-packages\shap\explainers\_deep\deep_tf.py", line 26, in custom_record_gradient
        out = tf_backprop._record_gradient("shap_"+op_name, inputs, attrs, results)

    AttributeError: Exception encountered when calling layer "dense_5" "                 f"(type Dense).
    
    module 'tensorflow.python.eager.backprop' has no attribute '_record_gradient'
    
    Call arguments received by layer "dense_5" "                 f"(type Dense):
      • inputs=tf.Tensor(shape=(91200, 11), dtype=float32)


In [ ]:
tf.__version__

In [ ]:
X

### Crossvalidation of the final classifier

In [ ]:
X_cv=np.concatenate([X_train_autoencoder_cnn_cnn_11_12,X_test_autoencoder_cnn_cnn_11_12])

In [ ]:
X_train_autoencoder_cnn_cnn_11_12.shape,X_test_autoencoder_cnn_cnn_11_12.shape

In [ ]:
y_cv=np.concatenate([y_train.values,y_test.values])
y_cv.shape

In [ ]:
kf = KFold(n_splits=5, shuffle=True)
ann_cv = keras.Sequential([
    keras.layers.Dense(128, input_shape=(11,), activation='relu'),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

ann_cv.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss="binary_crossentropy",
              metrics=['accuracy',tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])
  
for i1,(train_index , test_index) in enumerate(kf.split(X_cv)):
    print(f"Fold {i1+1}")
    X_train1 , X_test1 = X_cv[train_index,:],X_cv[test_index,:]
    y_train1 , y_test1 = y_cv[train_index] , y_cv[test_index]
     
    ann_cv.fit(X_train1,y_train1, epochs=60,validation_split=0.20)
    pred_values = ann_cv.predict(X_test1)
    cv_pred=[]
    for a in pred_values:
        if a[0]>=0.5:
            cv_pred.append(1)
        else:
            cv_pred.append(0)
        
    print("Accuracy: ",accuracy_score(cv_pred , y_test1))
    print(classification_report(cv_pred,y_test1))
    print(confusion_matrix(cv_pred,y_test1))
    print("#################################")

In [ ]:
fold1:1
    acc=0.93
    prec=0.92
    recall=0.91
    f1=0.92
fold:2
    acc=0.93
    prec=0.92
    recall=0.92
    f1=0.92
fold:3
    acc=0.93
    prec=0.95
    recall=0.91
    f1=0.93
fold:4
    acc=94
    prec=0.94
    recall=0.93
    f1=0.93
fold:5
    acc=95
    prec=0.94
    recall=0.94
    f1=0.94
    

#### showing the variation of the classifier w.r.t the bottleneck size

In [ ]:
input_dim = Input(shape = (19, 1))

for i in range(1,16):
    cnn_enc = layers.Conv1D(128, 19, activation='relu', padding='same')(input_dim)
    cnn_enc = layers.Conv1D(64, 19, activation='relu', padding='same')(cnn_enc)
    cnn_enc = layers.Conv1D(32, 19, activation='relu', padding='same')(cnn_enc)
    cnn_enc = layers.Conv1D(16, 19, activation='relu', padding='same')(cnn_enc)
    cnn_enc = layers.Flatten()(cnn_enc)

    bottleneck = Dense(i, activation='sigmoid')(cnn_enc)

    reshp= Reshape((i,1))(bottleneck)

    cnn_dec=layers.Conv1D(16, i, activation='relu',padding='same')(reshp)
    cnn_dec=layers.Conv1D(32, i, activation='relu',padding='same')(cnn_dec)
    cnn_dec=layers.Conv1D(64, i, activation='relu',padding='same')(cnn_dec)
    cnn_dec=layers.Conv1D(128, i, activation='relu',padding='same')(cnn_dec)

    dec_flat=layers.Flatten()(cnn_dec)

    # mlp=Dense(256,activation='relu')(dec_flat)
    mlp=Dense(19,activation='relu')(dec_flat)

    var_autoenc = keras.Model(input_dim, mlp)
    
    var_autoenc.compile(optimizer='adam', loss='mse',metrics=['accuracy','mae','MeanSquaredError'])
    var_autoenc.fit(X_train_selected_scaled_cnn,X_train_selected_scaled,epochs=100,validation_split=0.2,verbose=2)
    intermediate = Model(inputs=var_autoenc.input, outputs=var_autoenc.get_layer(index=6).output)
    X_train_vary = intermediate.predict(X_train_selected_scaled_cnn)
    X_test_vary = intermediate.predict(X_test_selected_scaled_cnn)
    print("#####################")
    print("Shape of train: ",X_train_vary.shape)
    print("Shape of train: ",X_test_vary.shape)
    
    ann_vary = keras.Sequential([
    keras.layers.Dense(128, input_shape=(i,), activation='relu'),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
    ])

    ann_vary.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
                  loss="binary_crossentropy",
                  metrics=['accuracy',tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])

    ann_vary.fit(X_train_vary, y_train, epochs=60,validation_split=0.25)

    p=ann_vary.predict(X_test_vary)
    p1=[]
    for j in p:
        if j[0]>=0.5:
            p1.append(1)
        else:
            p1.append(0)

    print(confusion_matrix(y_test,p1))
    print(classification_report(y_test,p1))

In [ ]:
#### Fix the above autoencoder and work with classifier

In [ ]:
#### Abalation study

In [ ]:
# with sigmoid activation
ann_cnn_cnn3 = keras.Sequential([
    keras.layers.Dense(128, input_shape=(11,), activation='sigmoid'),
    keras.layers.Dense(256, activation='sigmoid'),
    keras.layers.Dense(64, activation='sigmoid'),
    keras.layers.Dense(1, activation='sigmoid')
])

ann_cnn_cnn3.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss="binary_crossentropy",
              metrics=['accuracy',tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])

ann_cnn_cnn3.fit(X_train_autoencoder_cnn_cnn_11_12, y_train, epochs=60,validation_split=0.25)

pred3=ann_cnn_cnn3.predict(X_test_autoencoder_cnn_cnn_11_12)
pred4=[]
for j in pred3:
    if j[0]>=0.5:
        pred4.append(1)
    else:
        pred4.append(0)
        
print(confusion_matrix(y_test,pred4))
print(classification_report(y_test,pred4))

In [ ]:
# with leaky activation
ann_cnn_cnn4 = keras.Sequential([
    keras.layers.Dense(128, input_shape=(11,), activation=tf.keras.layers.LeakyReLU()),
    keras.layers.Dense(256, activation=tf.keras.layers.LeakyReLU()),
    keras.layers.Dense(64, activation=tf.keras.layers.LeakyReLU()),
    keras.layers.Dense(1, activation='sigmoid')
])

ann_cnn_cnn4.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss="binary_crossentropy",
              metrics=['accuracy',tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])

ann_cnn_cnn4.fit(X_train_autoencoder_cnn_cnn_11_12, y_train, epochs=60,validation_split=0.25)

pred4=ann_cnn_cnn4.predict(X_test_autoencoder_cnn_cnn_11_12)
pred5=[]
for j in pred4:
    if j[0]>=0.5:
        pred5.append(1)
    else:
        pred5.append(0)
        
print(confusion_matrix(y_test,pred5))
print(classification_report(y_test,pred5))

In [ ]:
# with selu activation
ann_cnn_cnn5 = keras.Sequential([
    keras.layers.Dense(128, input_shape=(11,), activation="selu"),
    keras.layers.Dense(256, activation="selu"),
    keras.layers.Dense(64, activation="selu"),
    keras.layers.Dense(1, activation='sigmoid')
])

ann_cnn_cnn5.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss="binary_crossentropy",
              metrics=['accuracy',tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])

ann_cnn_cnn5.fit(X_train_autoencoder_cnn_cnn_11_12, y_train, epochs=60,validation_split=0.25)

pred5=ann_cnn_cnn4.predict(X_test_autoencoder_cnn_cnn_11_12)
pred6=[]
for j in pred5:
    if j[0]>=0.5:
        pred6.append(1)
    else:
        pred6.append(0)
        
print(confusion_matrix(y_test,pred6))
print(classification_report(y_test,pred6))

In [ ]:
### Tuning the classifier head

In [ ]:
from kerastuner import RandomSearch
def hyper_model(hp):
    model=keras.Sequential() # Building the ANN
    for i in range(hp.Int('num_layers',2,6)):  # ANN must have hidden layers in between 2 to 6
        model.add(layers.Dense(units=hp.Int('units_'+str(i),min_value=8,max_value=256,step=8),# In each hidden layers the no of neurons must be between 32 to 256
                              activation='relu')) # activation function used
        model.add(layers.Dense(1,activation='sigmoid'))# Since it is a regressiom problem the output neuron should be 1 and the activation function should be softmax
        model.compile(optimizer=tf.keras.optimizers.Adam(      # Using Adam as optimizer and hypertune its parameter
                      hp.Choice('learning_rate',[1e-1,1e-2, 1e-3, 1e-4])), # LR for Adam
                      loss='binary_crossentropy',
                      metrics=['accuracy',tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])
        
    return model

In [ ]:
tuner = RandomSearch(
    hyper_model,
    objective='val_accuracy', # metric used for evaluating the ANN
    max_trials=10,
    executions_per_trial=5,
    directory='project',
    project_name='log')# there will be 10 trials and inside each trial there will be 5 execution so in total 30 trials

In [ ]:
tuner.search_space_summary()

In [ ]:
X_train_hype, X_test_hype, y_train_hype, y_test_hype =train_test_split(X_train_autoencoder_cnn_cnn_11_12,y_train,test_size=0.2,random_state=42)

In [ ]:
tuner.search(X_train_hype, y_train_hype,epochs=30,validation_data=(X_test_hype, y_test_hype))

In [ ]:
tuner.results_summary()

In [ ]:
### Plotting the AUC_ROC curves

## AutoEncoder CNN - MLP

In [ ]:
input_dim = Input(shape = (19, 1))

# cnn_enc=Sequential()
cnn_enc = layers.Conv1D(64, 19, activation='relu', padding='same')(input_dim)
cnn_enc = layers.Conv1D(32, 19, activation='relu', padding='same')(cnn_enc)
cnn_enc = layers.Conv1D(16, 19, activation='relu', padding='same')(cnn_enc)
cnn_enc = layers.Conv1D(8, 19, activation='relu', padding='same')(cnn_enc)
cnn_enc = layers.Flatten()(cnn_enc)

bottleneck = Dense(11, activation='sigmoid')(cnn_enc)

mlp=Dense(32,activation='relu')(bottleneck)
mlp=Dense(64,activation='relu')(mlp)
mlp=Dense(19,activation='relu')(mlp)

autoencoder_cnn_mlp = keras.Model(input_dim, mlp)

print(autoencoder_cnn_mlp.summary())


In [ ]:
autoencoder_cnn_mlp.compile(optimizer='adam', loss='mse',metrics=['accuracy','mae','MeanSquaredError'])

In [ ]:
autoencoder_cnn_mlp.fit(X_train_selected_scaled_cnn,X_train_selected_scaled,epochs=100,validation_split=0.2,verbose=2)

In [ ]:
plt.plot(autoencoder_cnn_mlp.history.history['accuracy'])
plt.plot(autoencoder_cnn_mlp.history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='lower right')
plt.show()

In [ ]:
plt.plot(autoencoder_cnn_mlp.history.history['loss'])
plt.plot(autoencoder_cnn_mlp.history.history['val_loss'])
plt.title('model accuracy')
plt.ylabel('Loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='lower right')
plt.show()

In [ ]:
intermediate_layer_model111 = Model(inputs=autoencoder_cnn_mlp.input, outputs=autoencoder_cnn_mlp.get_layer(index=6).output)
X_train_autoencoder_cnn_mlp_11_1 = intermediate_layer_model111.predict(X_train_selected_scaled_cnn)
X_test_autoencoder_cnn_mlp_11_1 = intermediate_layer_model111.predict(X_test_selected_scaled_cnn)
print("Shape of train: ",X_train_autoencoder_cnn_mlp_11_1.shape)
print("Shape of train: ",X_test_autoencoder_cnn_mlp_11_1.shape)

In [ ]:
ann_cnn_mlp = keras.Sequential([
    keras.layers.Dense(128, input_shape=(11,), activation='relu'),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

ann_cnn_mlp.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy',tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])

ann_cnn_mlp.fit(X_train_autoencoder_cnn_mlp_11_1, y_train, epochs=60,validation_split=0.25)

pred7=ann_cnn_mlp.predict(X_test_autoencoder_cnn_mlp_11_1)
pred71=[]
for j in pred7:
    if j[0]>=0.5:
        pred71.append(1)
    else:
        pred71.append(0)
        
print(confusion_matrix(y_test,pred71))
print(classification_report(y_test,pred71))

## Autoencoder MLP-MLP

In [ ]:
# AUTOENCODER======= 19->11->19

input_dim = Input(shape = (19, ))

mlp_mlp3 = Sequential()
mlp_mlp3.add(Dense(19,activation = 'relu',input_shape=[19]))
mlp_mlp3.add(Dense(11,activation = 'relu'))
mlp_mlp3.add(Dense(19,activation = 'relu'))

mlp_mlp3.compile(optimizer=Adam(0.001),metrics=['accuracy','mae','MeanSquaredError'],loss='mse')
print(mlp_mlp3.summary())

In [ ]:
mlp_mlp3.fit(x=X_train_selected_scaled, y=X_train_selected_scaled, epochs=100,validation_split=0.2)
plt.plot(mlp_mlp3.history.history['accuracy'])
plt.plot(mlp_mlp3.history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='lower right')
plt.show()
intermediate_layer_model_mlp1 = Model(inputs=mlp_mlp3.input, outputs=mlp_mlp3.get_layer(index=1).output)
X_train_autoencoder_mlp_mlp_11_1 = intermediate_layer_model_mlp1.predict(X_train_selected_scaled)
X_test_autoencoder_mlp_mlp_11_1 = intermediate_layer_model_mlp1.predict(X_test_selected_scaled)
print("Shape of train: ",X_train_autoencoder_mlp_mlp_11_1.shape)
print("Shape of train: ",X_test_autoencoder_mlp_mlp_11_1.shape)

In [ ]:
ann_mlp_mlp1 = keras.Sequential([
    keras.layers.Dense(128, input_shape=(11,), activation='relu'),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

ann_mlp_mlp1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy',tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])

ann_mlp_mlp1.fit(X_train_autoencoder_mlp_mlp_11_1, y_train, epochs=60,validation_split=0.25)

pred8=ann_mlp_mlp1.predict(X_test_autoencoder_mlp_mlp_11_1)
pred81=[]
for j in pred8:
    if j[0]>=0.5:
        pred81.append(1)
    else:
        pred81.append(0)
        
print(confusion_matrix(y_test,pred81))
print(classification_report(y_test,pred81))

In [ ]:
# AUTOENCODER======= 19->15->11->15->19

input_dim = Input(shape = (19, ))

mlp_mlp5 = Sequential()
mlp_mlp5.add(Dense(19,activation = 'relu',input_shape=[19]))
mlp_mlp5.add(Dense(15,activation = 'relu'))
mlp_mlp5.add(Dense(11,activation = 'relu'))
mlp_mlp5.add(Dense(15,activation = 'relu'))
mlp_mlp5.add(Dense(19,activation = 'relu'))

mlp_mlp5.compile(optimizer=Adam(0.001),metrics=['accuracy','mae','MeanSquaredError'],loss='mse')
print(mlp_mlp5.summary())

In [ ]:
mlp_mlp5.fit(x=X_train_selected_scaled, y=X_train_selected_scaled, epochs=100,validation_split=0.2)
plt.plot(mlp_mlp5.history.history['accuracy'])
plt.plot(mlp_mlp5.history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='lower right')
plt.show()
intermediate_layer_model_mlp2 = Model(inputs=mlp_mlp5.input, outputs=mlp_mlp5.get_layer(index=2).output)
X_train_autoencoder_mlp_mlp_11_2 = intermediate_layer_model_mlp2.predict(X_train_selected_scaled)
X_test_autoencoder_mlp_mlp_11_2 = intermediate_layer_model_mlp2.predict(X_test_selected_scaled)
print("Shape of train: ",X_train_autoencoder_mlp_mlp_11_2.shape)
print("Shape of train: ",X_test_autoencoder_mlp_mlp_11_2.shape)

In [ ]:
ann_mlp_mlp2 = keras.Sequential([
    keras.layers.Dense(128, input_shape=(11,), activation='relu'),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

ann_mlp_mlp2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy',tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])

ann_mlp_mlp2.fit(X_train_autoencoder_mlp_mlp_11_2, y_train, epochs=60,validation_split=0.25)

pred9=ann_mlp_mlp2.predict(X_test_autoencoder_mlp_mlp_11_2)
pred91=[]
for j in pred9:
    if j[0]>=0.5:
        pred91.append(1)
    else:
        pred91.append(0)
        
print(confusion_matrix(y_test,pred91))
print(classification_report(y_test,pred91))

In [ ]:
y_train = pd.Series(y, index=X_train_selected.index)
y_test=pd.Series(y,index=X_test_selected.index)

In [ ]:
X_train_selected.shape

In [ ]:
y_test.shape

In [ ]:
# CNN Encoder- CNN Decoder

def roc_curve_and_score(y_test, pred_proba):
    fpr, tpr, thresholds = roc_curve(y_test.ravel(), pred_proba.ravel())
    roc_auc = roc_auc_score(y_test.ravel(), pred_proba.ravel())
    return thresholds,fpr, tpr, roc_auc


plt.figure(figsize=(8, 6))
plt.rcParams.update({'font.size': 14})
plt.grid()
# CNN-CNN
thresholds_cc,fpr_cc, tpr_cc, roc_auc_cc = roc_curve_and_score(y_test, pred2)
plt.plot(fpr_cc, tpr_cc, color='darkorange', lw=2,
         label='CNN-CNN={0:.3f}'.format(roc_auc_cc))
# CNN-MLP
thresholds_cm,fpr_cm, tpr_cm, roc_auc_cm = roc_curve_and_score(y_test, pred7)
plt.plot(fpr_cm, tpr_cm, color='green', lw=2,
         label='CNN-MLP={0:.3f}'.format(roc_auc_cm))
# MLP-MLP
thresholds_mm,fpr_mm, tpr_mm, roc_auc_mm = roc_curve_and_score(y_test, pred8)
plt.plot(fpr_mm, tpr_mm, color='crimson', lw=2,
         label='MLP-MLP={0:.3f}'.format(roc_auc_mm))
plt.plot([0, 1], [0, 1], color='navy', lw=1, linestyle='--')
plt.legend(loc="lower right")
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.show()

In [ ]:
y_test.shape

In [ ]:
auc_for_cnncnn=pd.concat([pd.Series(thresholds_cc),pd.Series(fpr_cc),pd.Series(tpr_cc)],axis=1)

In [ ]:
auc_for_cnncnn.columns=['thresholds','fpr','tpr']

In [ ]:
auc_for_cnncnn.head()

In [ ]:
auc_for_cnncnn.to_excel("auc_roc_comparison.xlsx")

In [ ]:
thresholding_list=[]

for thres in thresholds_cc:
    row=[]
    pred_thres=[]
    for j in pred2:
        if j[0]>=thres:
            pred_thres.append(1)
        else:
            pred_thres.append(0)
    row.append(thres)
    row.append(accuracy_score(y_test,pred_thres))
    row.append(f1_score(y_test,pred_thres))
    row.append(precision_score(y_test,pred_thres))
    row.append(recall_score(y_test,pred_thres))
    thresholding_list.append(row)

thresholding_df=pd.DataFrame(thresholding_list)
thresholding_df.columns=['Threshold','Accuracy','F1 Score','Precision','Recall']

In [ ]:
thresholding_df.head()

In [ ]:
## Sorting the dataframe by F1 Score

In [ ]:
thresholding_df.nlargest(n=20,columns='F1 Score')

In [ ]:
### classifer after hypertuned

In [ ]:
# with leaky activation
hype_ann = keras.Sequential([
    keras.layers.Dense(160, input_shape=(11,), activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

hype_ann.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
              loss="binary_crossentropy",
              metrics=['accuracy',tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])

hype_ann.fit(X_train_autoencoder_cnn_cnn_11_12, y_train, epochs=60,validation_split=0.25)

pred_hype=hype_ann.predict(X_test_autoencoder_cnn_cnn_11_12)
pred_hype1=[]
for j in pred_hype:
    if j[0]>=0.5:
        pred_hype1.append(1)
    else:
        pred_hype1.append(0)
        
print(confusion_matrix(y_test,pred_hype1))
print(classification_report(y_test,pred_hype1))